In [6]:
from compare import *
#from lava_generator import *
from xai import latent_space_display_mark

In [7]:
vae = VAE.load("trained_models/VAE")
cnn = load_model("trained_models/CNN/classifier.h5")
xai = xai_model(vae.decoder, cnn, input_shape=(10,))

# input images
# samples_test, sample_labels_test = load_samples_for_test_folder(img_dir='./contrib/DLFuzz/MNIST/seeds_50')
samples_test, sample_labels_test = load_samples_for_test(200)

# prepare
view_samples = samples_test
view_sample_labels = sample_labels_test
# view_samples = samples
# view_sample_labels = sample_labels

x_view = np.reshape(view_samples, (-1, 784))
y_view_onehot = tf.one_hot(tf.constant(view_sample_labels), depth=10).numpy()
h_view = vae.encoder.predict(x_view)

K.set_learning_phase(0)
dlfuzz = DLFuzz(cnn)

7/7 [==============================] - 0s 2ms/step


In [10]:
def generate_adversarial_with_deep_optimizer(h_view, label, learning_rate=0.01, num_iterations=100):
    # load models
    vae = VAE.load("trained_models/VAE")
    cnn = load_model("trained_models/CNN/classifier.h5")
    xai = xai_model(vae.decoder, cnn, input_shape=(10,))

    # prepare data
    x = np.array([h_view])
    y = np.array([label])


    # Use Adam optimizer
    optimizer = tf.optimizers.Adam(learning_rate)
    for iteration in range(num_iterations):
        # Compute gradient using the customized method
        gradient = gradient_of_x(x, y, xai)
        #x_var = tf.Variable(x, dtype=tf.float32)
        optimizer.apply_gradients([(gradient, x)])

        # Decode the perturbed image and check if misclassified
        decoded_image = vae.decoder.predict(x).reshape((28, 28, 1))

        # Check if the decoded image is misclassified by the CNN
        prediction = cnn.predict(np.array([decoded_image]))[0]
        predicted_label = np.argmax(prediction)

        if predicted_label != np.argmax(y):
            print(f"Misclassification achieved at iteration {iteration}")
            break

    return [decoded_image, predicted_label]


In [11]:
# calculate latent variant image
i = 3

image_org = np.array([view_samples[i]], dtype="float32")
label_org = view_sample_labels[i]

x = np.array([h_view[i]])
y = np.array([y_view_onehot[i]])

# Compute initial gradient
g = gradient_of_x(x, y, xai)

g_npy = np.squeeze(g.numpy())

# Parameters for gradient descent
alpha = 0.01  # Learning rate
num_iterations = 10  # Number of iterations for gradient descent

# Adversarial example generation
[decoded_image, prediction] = generate_adversarial_with_deep_optimizer(h_view[1], y_view_onehot[1], alpha, num_iterations)

image_gen_lava = vae.decoder.predict(x)
images = [image_org, decoded_image]
titles = [f'image_org_{label_org}', f'image_gen_lava_{prediction}']
plot_image_comparison(images, titles)

AttributeError: 'numpy.ndarray' object has no attribute '_unique_id'